In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
import tensorflow as tf
import datetime


In [3]:
model_best = tf.keras.models.load_model("model_best.h5")

model_best.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 1000)              6000      
                                                                 
 dense_28 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_29 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_30 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_31 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_32 (Dense)            (None, 6)                 6006      
                                                                 
Total params: 4,016,006
Trainable params: 4,016,006
No

In [ ]:
data_train = pd.read_csv("train_dataset_train.csv")

In [3]:
data_train = pd.read_csv("train_dataset_train.csv")

In [4]:
data_train.head(3)

,id,Easting,Northing,Height,Reflectance,Class
0,2321251,431696.5375,6.032319e+06,69.2226,-11.14,0
1,3515173,431710.3835,6.032291e+06,68.9711,-15.16,3
2,2320295,431696.8099,6.032322e+06,69.2453,-13.59,0


In [5]:
data_y = data_train["Class"]
encod_y = LabelBinarizer()
Y_train = encod_y.fit_transform(data_y)


In [6]:
from sklearn.preprocessing import StandardScaler
def get_train_dataset_x(data_x):

  data_X = data_x.copy()
  norm = MinMaxScaler()
  standart = StandardScaler()
  res = norm.fit_transform(data_X)
  res = standart.fit_transform(res)
  X_train = res

  return X_train, norm, standart

In [7]:
def get_test_dataset_x(data_x, norm, pca):

  data_X = data_x.copy()
  res = norm.transform(data_X)
  res = pca.transform(res)
  X_test = res

  return X_test

In [8]:
data_x = data_train[data_train.columns[:-1]]
X_train, norm, pca = get_train_dataset_x(data_x)
X_train

array([[-0.39952713, -0.34377509,  0.39363582, -0.07885515, -0.13495634],
       [ 0.2861392 ,  0.12128049, -0.57336233, -0.18965378, -1.02526729],
       [-0.40007616, -0.3346258 ,  0.49395797, -0.06885463, -0.67755879],
       ...,
       [ 1.50728161,  0.95142117, -0.50097917, -0.19344252,  0.05107878],
       [ 0.46913334, -0.15680905,  0.34321815, -0.05035148,  0.13080812],
       [-1.60603817,  0.16597567,  1.07204383, -1.08987615,  1.22487183]])

In [9]:
X_train, norm, pca = get_train_dataset_x(data_x)


In [13]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers.schedules import ExponentialDecay
initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)
opt = Adam(learning_rate=lr_schedule)
met = Recall()
model_best = Sequential()
model_best.add(Dense(1000, activation="relu", input_shape = (X_train.shape[1],)))
model_best.add(Dense(1000, activation="relu"))
model_best.add(Dense(1000, activation="relu"))
model_best.add(Dense(1000, activation="relu"))
model_best.add(Dense(1000, activation="relu"))
model_best.add(Dense(Y_train.shape[1], activation="softmax"))
model_best.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[met])

In [11]:
print(X_train.shape[1])
model_best.summary()

5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              6000      
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_2 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_3 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_4 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_5 (Dense)             (None, 6)                 6006      
                                                                 
Total params: 4,016,006
Trainable params: 4,016,006
No

In [21]:
model_best = tf.keras.models.load_model("model_standart.h5")

In [14]:
history_best = model_best.fit(X_train, Y_train, epochs=5, validation_split = 0.01, batch_size=2500, verbose = 1)

Epoch 1/5
1672/1672 [==============================] - 33s 19ms/step - loss: 0.1685 - recall_2: 0.9484 - val_loss: 0.0875 - val_recall_2: 0.9644

KeyboardInterrupt: 

In [75]:
test = pd.read_csv("test_dataset_test.csv")
X_test = get_test_dataset_x(test[test.columns[1:]], norm, pca)
Y_test = model_best.predict(X_test, batch_size = 100000, verbose=1)

19/19 [==============================] - 3s 182ms/step


In [76]:
classif = encod_y.inverse_transform(Y_test)
id = test["id"].to_numpy()
res = list(zip(id,classif))
out = pd.DataFrame(res, columns=["id", "Class"])
out.to_csv(f"result_standart.csv", index=False)

In [73]:
model_best.save(f"model_standart.h5")


